In [32]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
# import keras
from keras.models import Sequential
from keras.layers import Dense
from keras_tuner import RandomSearch
from tensorflow.keras.optimizers import Adam

In [33]:
#load dataset
data=pd.read_csv("Real_Combine.csv")
data.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [34]:
data.columns

Index(['T', 'TM', 'Tm', 'SLP', 'H', 'VV', 'V', 'VM', 'PM 2.5'], dtype='object')

In [57]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1093 entries, 0 to 1092
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   T       1093 non-null   float64
 1   TM      1093 non-null   float64
 2   Tm      1093 non-null   float64
 3   SLP     1093 non-null   float64
 4   H       1093 non-null   float64
 5   VV      1093 non-null   float64
 6   V       1093 non-null   float64
 7   VM      1093 non-null   float64
 8   PM 2.5  1092 non-null   float64
dtypes: float64(9)
memory usage: 77.0 KB


In [35]:
#making dependent and independent variable
x=data.iloc[:,0:8]#independent variable
y=data.iloc[:,8]

In [60]:
data[(data["PM 2.5"]>108) & (data["PM 2.5"] < 110)]

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
23,18.3,23.2,12.7,1011.2,75.0,1.9,10.0,22.2,109.333333
226,22.4,32.5,14.5,1007.2,68.0,2.1,5.4,11.1,109.458333
228,25.2,32.7,19.1,1010.7,64.0,1.9,2.8,11.1,108.208333
249,31.8,38.6,24.2,1004.3,54.0,1.9,1.9,7.6,108.333333
412,28.6,36.5,20.3,1012.4,59.0,1.8,9.3,55.4,109.750000
574,17.4,24.4,10.1,1018.9,61.0,2.3,6.1,11.1,109.333333
605,33.6,40.0,23.0,1003.8,27.0,2.4,9.8,18.3,109.625000
755,17.4,24.4,10.1,1018.9,61.0,2.3,6.1,11.1,109.333333
786,33.6,40.0,23.0,1003.8,27.0,2.4,9.8,18.3,109.625000
936,17.4,24.4,10.1,1018.9,61.0,2.3,6.1,11.1,109.333333


In [36]:
#train_test_split
x_train , x_test , y_train , y_test=train_test_split(x , y , test_size=0.2 , random_state=0)
print(x_test , x_train , y_test , y_train)

         T    TM    Tm     SLP     H   VV     V    VM
784   31.6  38.0  23.2  1002.6  30.0  3.1  18.7  35.2
741   11.9  15.7   7.5  1018.9  86.0  0.8   6.1  11.1
747   15.9  22.4   8.4  1017.6  66.0  2.4   7.2  16.5
986   32.9  39.0  27.4   996.6  50.0  2.4   9.4  27.8
479   31.6  35.5  25.4  1001.9  63.0  1.9   5.4  11.1
...    ...   ...   ...     ...   ...  ...   ...   ...
655   31.9  35.0  28.3  1001.3  72.0  2.1   2.8   9.4
1087  16.3  23.0   6.6  1016.9  61.0  1.0   9.4  20.6
869   30.3  35.0  24.7  1007.3  62.0  2.3   7.6  18.3
665   29.9  34.0  25.5  1000.2  80.0  1.4   3.3  11.1
674   27.8  31.0  25.2  1001.3  88.0  2.4   4.6   7.6

[219 rows x 8 columns]          T    TM    Tm     SLP     H   VV     V    VM
863   32.5  37.2  25.3  1003.0  59.0  3.1   1.7   7.6
422   26.4  32.5  20.0  1012.8  68.0  2.6   3.0  11.1
342   23.4  31.2  15.0  1013.4  58.0  1.1   4.1  14.8
188   11.9  20.7   4.4  1017.9  78.0  0.8   1.9   7.6
934   17.2  26.0  11.5  1014.7  65.0  2.3   5.7  14.8
... 

In [37]:
y_train = np.where(np.isnan(y_train), np.nanmean(y_train), y_train)

In [38]:
from sklearn.preprocessing import StandardScaler
import numpy as np

# Convert y to NumPy array and make 2D
y_train = np.array(y_train).reshape(-1, 1)
y_test = np.array(y_test).reshape(-1, 1)

# Feature scaler
X_scaler = StandardScaler()
X_train_scaled = X_scaler.fit_transform(x_train)
X_test_scaled = X_scaler.transform(x_test)

# Target scaler
y_scaler = StandardScaler()
y_train_scaled = y_scaler.fit_transform(y_train)
y_test_scaled = y_scaler.transform(y_test)

# Check NaNs or Infs
print(np.isnan(X_train_scaled).sum(), np.isnan(y_train_scaled).sum())
print(np.isinf(X_train_scaled).sum(), np.isinf(y_train_scaled).sum())


0 0
0 0


In [39]:
#building a model

def build_model(hp):
    model=Sequential()
    model.add(Dense(units=hp.Int('units_input', min_value=32 , max_value=256 , step=32),
                     activation=hp.Choice('activation _input',['relu' , 'tanh']),
                     input_dim=X_train_scaled.shape[1]))
    
    for i in range(hp.Int('num_layers',1,3)):
        model.add(Dense(units=hp.Int(f'units_{i}', min_value=32 , max_value=256 , step=32),
                        activation=hp.Choice(f'activation{i}',['relu' , 'tanh'])))
    model.add(Dense(units=1 , activation='linear'))
    #compiling model
    model.compile(optimizer=Adam(hp.Choice('learning_rate', [1e-3, 1e-4])) , loss='mean_squared_error' ,metrics=['mean_absolute_error'])
    return model

In [40]:
tuner=RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=2,
    directory='my_dir',
    project_name='air_quality_ann',
    overwrite=True
)
tuner.search(X_train_scaled , y_train_scaled, validation_split=0.1, epochs=50, batch_size=16)

Trial 2 Complete [00h 00m 15s]
val_loss: 0.5305104851722717

Best val_loss So Far: 0.4233362078666687
Total elapsed time: 00h 00m 31s


In [42]:
best_model = tuner.get_best_models(num_models=1)

C:\my coding journey\deep_learning\deep\Lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [43]:
best_model = best_model[0]


In [61]:
# ---------------------------
# Take user input and predict
# ---------------------------
print("Enter air quality features to predict PM2.5:")
user_input = []
for col in data.drop("PM 2.5", axis=1).columns:
    value = float(input(f"{col}: "))
    user_input.append(value)

# Convert to numpy array and scale
user_input = np.array(user_input).reshape(1, -1)
user_input_scaled = X_scaler.transform(user_input)

# Make prediction
predicted_pm25_scaled = best_model.predict(user_input_scaled)
predicted_pm25 = y_scaler.inverse_transform(predicted_pm25_scaled)
print(f"Predicted PM2.5 value: {predicted_pm25[0][0]:.2f}")

Enter air quality features to predict PM2.5:


T:  18.3
TM:  23.2
Tm:  12.7
SLP:  1011.2
H:  75.0
VV:  1.9
V:  10.0
VM:  22.2


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step

C:\my coding journey\deep_learning\deep\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
Predicted PM2.5 value: 122.44


In [ ]:
18.3	23.2	12.7	1011.2	75.0	1.9	10.0	22.2	